# Análisis y preprocesamiento de los datos

Se explorarán, en pequeños experimentos, distintas formas de representación de los datos del corpus WiNER (Ghaddar y Langlais 2017) para utilizarlos en la tarea de reconocimiento de entidades nombradas. Para esto se exploran distintas combinaciones de vectores de palabras como representación de una instancia de entrenamiento (Iacobacci et al. 2016).

### Descripción del Corpus WiNER

* Documents.tar.bz2 : este archivo contiene 3256489 (TODO: check this!) artículos de Wikipedia repartidos en 3225 archivos. Cada archivo contiene aproximadamente 1000 artículos nombrados por sus respectivos IDs. Los artículos están indexados por su wikiID seguidos de oraciones (una por línea), donde las palabras son remplazadas por sus ids.

<br>
* document.vocab : este archivo contiene el mapeo de palabras (case sensitive); el formato es: 
           palabra <espacio> #ocurrencias
  El ID de cada palabra es el número de línea en la cual ocurre

In [1]:
import pandas as pd
import numpy as np
import time
import gensim
from gensim.models import Word2Vec, KeyedVectors

### Cargamos los datos

In [2]:
word_mapping = pd.read_csv('./corpus_WiNER/document.vocab', sep=' ', header=None,
                           keep_default_na=False) # con esto evito que el parser trate como 
                                                  # nan value a algunas palabras.
word_mapping.columns = ['word', 'frequency']

In [3]:
print(word_mapping.shape)
word_mapping.head()

(6931358, 2)


,word,frequency
0,the,73411953
1,",",68044881
2,.,54241850
3,of,40550466
4,and,34602894


In [4]:
len(word_mapping['word'].unique())

6931358

### Cargamos los artículos del Documento "0"

In [5]:
doc_0 = pd.read_csv('./corpus_WiNER/Documents/0', sep='ID', engine='python', header=None, names=['sentence', 'art_ID'])

In [6]:
doc_0.shape

(130209, 2)

In [7]:
doc_0.head()

,sentence,art_ID
0,NaN,431388.0
1,650 5590 753942 12 189 243 1 2039 47 257 682 1...,NaN
2,34 23207 1523 4 57193 15 10777 14353 4 157019 ...,NaN
3,418 0 1858 101 0 1322 1 5590 18 860 729 14 92 ...,NaN
4,650 5590 1523 8 1136 15 11301 575 1156 1 2746 ...,NaN


Vamos a asociarle a cada oración el ID del artículo al cual pertenece.

Es importante recordar que el orden de las oraciones está dado por los índices del dataframe.

In [8]:
art_ID_list = doc_0['art_ID'].tolist()
art_ID = 0
for idx, elem in enumerate(art_ID_list):
    if not np.isnan(elem):
        art_ID = elem
    else:
        art_ID_list[idx] = art_ID
doc_0['art_ID'] = art_ID_list

Removemos ahora las filas con NaN que contenian los ID de los artículos inicialmente.

In [9]:
doc_0 = doc_0.dropna()

In [10]:
print('El documento contiene {} oraciones.'.format(doc_0.shape[0]))
print('El documento contiene {} artículos'.format(len(doc_0['art_ID'].unique())))

El documento contiene 128395 oraciones.
El documento contiene 1814 artículos


Hacemos que cada sentencia sea una lista de palabras codificadas

In [11]:
doc_0['sentence'] = doc_0['sentence'].map(lambda x: x.split(' '))

In [12]:
doc_0.head()

,sentence,art_ID
1,"[650, 5590, 753942, 12, 189, 243, 1, 2039, 47,...",431388.0
2,"[34, 23207, 1523, 4, 57193, 15, 10777, 14353, ...",431388.0
3,"[418, 0, 1858, 101, 0, 1322, 1, 5590, 18, 860,...",431388.0
4,"[650, 5590, 1523, 8, 1136, 15, 11301, 575, 115...",431388.0
5,"[152, 642, 16, 8143, 23122, 20, 0, 662955, 16,...",431388.0


Nos quedamos con un artículo

In [13]:
article = doc_0[doc_0.art_ID == 1000]
article.head()

,sentence,art_ID
2756,"[63903, 24730, 9, 7, 3035, 4104, 7584, 1, 355,...",1000.0
2757,"[3654, 16, 2502, 39413, 1, 24730, 9, 44, 3, 99...",1000.0
2758,"[24730, 35, 49, 3521, 15, 575, 1, 15, 2440, 1,...",1000.0
2759,"[152, 112, 8, 2037, 20, 52, 54, 3035, 17572, 3...",1000.0
2760,"[64, 62, 5787, 1132, 15, 0, 144, 24730, 1312, ...",1000.0


Reconstruimos la primera oración del artículo

In [14]:
def sentence_decoder(sentence, word_mapping):
    dec_sentence = []
    for idx in sentence:
        mapped_w = word_mapping.loc[np.int(idx), 'word']
        dec_sentence.append(mapped_w)

    return dec_sentence 

In [15]:
sentence = article.loc[2756, 'sentence']

In [17]:
dec_sentence = sentence_decoder(sentence, word_mapping)
' '.join(dec_sentence)

'Hercule Poirot is a fictional Belgian detective , created by Agatha Christie .'

### Word embeddings utilizando el modelo pre-entrenado word2vec de Google

Utilizaremos la librería Gensim https://radimrehurek.com/gensim/

Cargamos Google's pre-trained Word2Vec model.

Utilizando KeyedVectors para cargar el modelo tiene la desventaja de que no se puede seguir entrenando. Pero es más eficiente que utilizar gensim.models.Word2Vec
https://radimrehurek.com/gensim/models/keyedvectors.html#module-gensim.models.keyedvectors

In [20]:
start = time.time()
# model = KeyedVectors.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)
# model.save('./models/word2vecGoogle.model')
model = KeyedVectors.load('./models/word2vecGoogle.model')
end = time.time()
print('demora: {}'.format(end-start))
# model = Word2Vec.load_word2vec_format('./models/GoogleNews-vectors-negative300.bin', binary=True)

demora: 21.5048611164093


In [35]:
len(model.vectors)

3000000